`parse_results.ipynb`: parse results from database into CSV.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()
from auctions.webutils import *

from open_spiel.python.examples.ubc_cma import get_results, get_game_info, get_algorithm_from_run, analyze_samples

# TODO: run this analysis in the jobs
from open_spiel.python.algorithms.exploitability import nash_conv
from open_spiel.python.examples.ubc_decorators import ModalAgentDecorator

In [3]:
def experiment_to_csv(experiments, csv_path):
    runs = []
    for experiment in experiments:
        runs += Experiment.objects.get(name=experiment).equilibriumsolverrun_set.all()
    print(f"Found {len(runs)} runs")

    records = []
    for run in tqdm(runs):    
        game = run.game.load_as_spiel()
        record = {
            'run_name': run.name,
            'game_name': run.game.name, 
            'seed': run.config.get('seed'), 
            'config': run.get_config_name(),
            'alg': get_algorithm_from_run(run),
        }
        
        record.update(get_game_info(game, run.game))  
        
        record['no_error'] = False
        records.append(record) # Put it here so you see the False's in the display
            
        try:
            game, final_checkpoint, _ = get_results(run, load_policy=False)
        except Exception as e:
            print(f"Failed to get_results for run {run.name}. Error: {e}")
            continue
        
        try:
            record['t'] = final_checkpoint.t
            record['walltime'] = run.walltime()

            for (evaluation, prefix) in [
                (final_checkpoint.get_modal_eval(), ''),
                (final_checkpoint.get_straightforward_eval(), 'straightforward_')
            ]:
                record[f'{prefix}nash_conv'] = evaluation.nash_conv
                record[f'{prefix}rewards'] = evaluation.mean_rewards
                record[f'{prefix}nash_conv_frac'] = evaluation.nash_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.nash_conv) else np.nan
                record[f'{prefix}heuristic_conv'] = evaluation.heuristic_conv
                record[f'{prefix}heuristic_conv_frac'] = evaluation.heuristic_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.heuristic_conv) else np.nan

                for i in range(game.num_players()):
                    record[f'{prefix}rewards_{i}'] = evaluation.mean_rewards[i]
                    record[f'{prefix}nc_player_improvements_{i}'] = evaluation.nash_conv_player_improvements[i] if not pd.isnull(evaluation.nash_conv) else np.nan
                    record[f'{prefix}nc_player_improvements_frac_{i}'] = (evaluation.nash_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.nash_conv) else np.nan
        
                    record[f'{prefix}hc_player_improvements_{i}'] = evaluation.heuristic_conv_player_improvements[i] if not pd.isnull(evaluation.heuristic_conv) else np.nan
                    record[f'{prefix}hc_player_improvements_frac_{i}'] = (evaluation.heuristic_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.heuristic_conv) else np.nan

                sample_metrics = analyze_samples(evaluation.samples, game)
                sample_metrics = {f'{prefix}{k}': sample_metrics[k] for k in sample_metrics} 
                record.update(**sample_metrics)

            record.update(**final_checkpoint.get_modal_eval().validation_info)
            record['no_error'] = True
        except Exception as e:
            print(f"Failed to parse results for run {run}. Error: {e}")
            # raise e
            # break
            # import traceback
            # print(traceback.format_exc())

    print(f'Parsed results for {len(records)} runs.')

    df = pd.DataFrame.from_records(records)
    df.to_csv(csv_path, index=False)

In [9]:
experiment_to_csv(['feb8_3p_5supply_v1'], 'results/feb8_3p_5supply_v1.csv')

Found 200 runs


 57%|█████▊    | 115/200 [03:52<01:13,  1.15it/s]

Failed to parse results for run feb8_3p_5supply_feb8_3p_5supply_0_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-102 (feb8_3p_5supply_v1). Error: Evaluation matching query does not exist.
Failed to parse results for run feb8_3p_5supply_feb8_3p_5supply_0_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-103 (feb8_3p_5supply_v1). Error: Evaluation matching query does not exist.
Failed to parse results for run feb8_3p_5supply_feb8_3p_5supply_0_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-104 (feb8_3p_5supply_v1). Error: Evaluation matching query does not exist.


 58%|█████▊    | 117/200 [03:52<00:46,  1.80it/s]

Failed to parse results for run feb8_3p_5supply_feb8_3p_5supply_0_base_dev1000_rho1_t3_tie_break-ppo_jun8_23ppo_76-100 (feb8_3p_5supply_v1). Error: Evaluation matching query does not exist.
Failed to parse results for run feb8_3p_5supply_feb8_3p_5supply_0_base_dev1000_rho1_t3_tie_break-ppo_jun8_23ppo_76-101 (feb8_3p_5supply_v1). Error: Evaluation matching query does not exist.
Failed to parse results for run feb8_3p_5supply_feb8_3p_5supply_0_base_dev1000_rho1_t3_tie_break-ppo_jun8_23ppo_76-102 (feb8_3p_5supply_v1). Error: Evaluation matching query does not exist.


...

Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_1_base_dev1000_rho0_t3_tie_break-ppo_jun8_23ppo_76-103. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_1_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-100. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_1_base_dev1000_rho0_t3_tie_break-ppo_jun8_23ppo_76-104. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_1_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-101. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_1_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-102. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_1_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-103. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_1_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-104. Error: None final checkpoint?
Failed to get_

 93%|█████████▎| 186/200 [03:53<00:00, 77.22it/s]

Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_2_base_dev1000_rho1_t3_tie_break-ppo_jun8_23ppo_76-103. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho0_t3-ppo_jun8_23ppo_76-100. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho0_t3-ppo_jun8_23ppo_76-102. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_2_base_dev1000_rho1_t3_tie_break-ppo_jun8_23ppo_76-104. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho0_t3-ppo_jun8_23ppo_76-101. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho0_t3-ppo_jun8_23ppo_76-103. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho0_t3-ppo_jun8_23ppo_76-104. Error: None final checkpoint?
Failed to get_

100%|██████████| 200/200 [03:53<00:00,  1.17s/it]

Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_4_base_dev1000_rho0_t3_tie_break-ppo_jun8_23ppo_76-103. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_4_base_dev1000_rho0_t3_tie_break-ppo_jun8_23ppo_76-104. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_4_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-100. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_4_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-101. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_4_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-102. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_4_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-103. Error: None final checkpoint?
Failed to get_results for run feb8_3p_5supply_feb8_3p_5supply_4_base_dev1000_rho1_t3-ppo_jun8_23ppo_76-104. Error: None final checkpoint?
Failed to get_

In [7]:
experiment_to_csv(['feb8_3p_7supply_v2'], 'results/feb8_3p_7supply_v2.csv')

Found 200 runs


100%|██████████| 200/200 [06:43<00:00,  2.02s/it]

Parsed results for 200 runs.


In [6]:
experiment_to_csv(['feb8_v1'], 'results/feb8_v1.csv')

Found 660 runs


100%|██████████| 660/660 [15:47<00:00,  1.44s/it]


Parsed results for 660 runs.


In [5]:
experiment_to_csv(['feb7_v1'], 'results/feb7_v1.csv')

Found 400 runs


  4%|▎         | 14/400 [00:19<08:55,  1.39s/it]

Failed to get_results for run feb4_4t_feb4_4t_0_base_dev1000_rho0_t4-ppo_jun8_23ppo_76-101. Error: None final checkpoint?


 14%|█▎        | 54/400 [01:14<08:18,  1.44s/it]

Failed to parse results for run feb4_4t_feb4_4t_1_base_dev1000_rho0_t4-ppo_jun8_23ppo_76-101 (feb7_v1). Error: Evaluation matching query does not exist.


...

Failed to get_results for run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-101. Error: None final checkpoint?


 95%|█████████▌| 381/400 [08:45<00:26,  1.42s/it]

Failed to get_results for run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4_tie_break-ppo_jun8_23ppo_76-101. Error: None final checkpoint?


100%|██████████| 400/400 [09:11<00:00,  1.38s/it]

Parsed results for 400 runs.


In [4]:
experiment_to_csv(['feb6_v2'], 'results/feb6_v2.csv')

Found 220 runs


  4%|▍         | 9/220 [00:13<05:07,  1.46s/it]

Failed to get_results for run feb4_4t_feb4_4t_1_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-104. Error: None final checkpoint?


  5%|▌         | 12/220 [00:16<04:19,  1.25s/it]

Failed to get_results for run feb4_4t_feb4_4t_1_base_dev1000_rho0_t4_tie_break-ppo_jun8_23ppo_76-109. Error: None final checkpoint?


...

Failed to get_results for run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-107. Error: None final checkpoint?


 95%|█████████▍| 208/220 [04:38<00:14,  1.23s/it]

Failed to get_results for run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-110. Error: None final checkpoint?


100%|██████████| 220/220 [04:48<00:00,  1.31s/it]

Failed to get_results for run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4_tie_break-ppo_jun8_23ppo_76-107. Error: None final checkpoint?
Failed to get_results for run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4_tie_break-ppo_jun8_23ppo_76-108. Error: None final checkpoint?
Failed to get_results for run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4_tie_break-ppo_jun8_23ppo_76-109. Error: None final checkpoint?
Failed to get_results for run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4_tie_break-ppo_jun8_23ppo_76-110. Error: None final checkpoint?
Parsed results for 220 runs.


In [8]:
experiment_to_csv(['feb6_v1'], 'results/feb6_v1.csv')

Found 220 runs


  0%|          | 1/220 [00:01<04:11,  1.15s/it]

Failed to parse results for run feb4_4t_feb4_4t_1_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-105 (feb6_v1). Error: update() argument after ** must be a mapping, not NoneType
Failed to get_results for run feb4_4t_feb4_4t_1_base_dev1000_rho0_t4_tie_break-ppo_jun8_23ppo_76-109. Error: None final checkpoint?


  1%|▏         | 3/220 [00:02<02:37,  1.38it/s]

Failed to parse results for run feb4_4t_feb4_4t_1_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-101 (feb6_v1). Error: update() argument after ** must be a mapping, not NoneType


...

Failed to parse results for run feb4_4t_feb4_4t_1_base_dev1000_rho1_t4_tie_break-ppo_jun8_23ppo_76-100 (feb6_v1). Error: update() argument after ** must be a mapping, not NoneType


  3%|▎         | 6/220 [00:05<03:37,  1.02s/it]

Failed to parse results for run feb4_4t_feb4_4t_1_base_dev1000_rho1_t4-ppo_jun8_23ppo_76-108 (feb6_v1). Error: update() argument after ** must be a mapping, not NoneType


  3%|▎         | 6/220 [00:06<04:01,  1.13s/it]


KeyboardInterrupt: 

In [ ]:
# only PPO
experiment_to_csv(['feb5_v2'], 'results/feb5_v2.csv')

In [6]:
experiment_to_csv(['feb5_v2'], 'results/feb5_v2.csv')

Found 660 runs


 69%|██████▉   | 455/660 [08:23<03:02,  1.12it/s]

Failed to parse results for run feb4_4t_feb4_4t_0_base_dev1000_rho0_t4_tie_break-ppo_jun8_23ppo_76-104 (feb5_v2). Error: Evaluation matching query does not exist.


 89%|████████▉ | 590/660 [11:10<01:16,  1.09s/it]

Failed to parse results for run feb4_4t_feb4_4t_3_base_dev1000_rho0_t4_tie_break-ppo_jun8_23ppo_76-106 (feb5_v2). Error: Evaluation matching query does not exist.


100%|██████████| 660/660 [12:30<00:00,  1.14s/it]


Parsed results for 660 runs.


In [5]:
experiment_to_csv(['feb4_v1'], 'results/feb4_v1.csv')

Found 800 runs


  2%|▏         | 14/800 [00:00<00:05, 133.76it/s]

Skipping run feb4_3t_feb4_3t_2_base_dev1000_rho0_t3_tie_break-cfr_10external_plus_linear-101 because of error None final checkpoint?
Skipping run feb4_1t_feb4_1t_0_base_dev1000_rho0_t1-cfr_10external_plus_linear-100 because of error None final checkpoint?
Skipping run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4-cfr_10external_plus_linear-104 because of error None final checkpoint?
Skipping run feb4_1t_feb4_1t_0_base_dev1000_rho0_t1-cfr_10external_plus_linear-101 because of error None final checkpoint?
Skipping run feb4_1t_feb4_1t_1_base_dev1000_rho1_t1_tie_break-cfr_10external_plus_linear-104 because of error None final checkpoint?
Skipping run feb4_2t_feb4_2t_2_base_dev1000_rho1_t2-cfr_10external_plus_linear-100 because of error None final checkpoint?
Skipping run feb4_2t_feb4_2t_2_base_dev1000_rho1_t2-cfr_10external_plus_linear-101 because of error None final checkpoint?
Skipping run feb4_3t_feb4_3t_0_base_dev1000_rho1_t3-cfr_10external_plus_linear-100 because of error None final checkpoi

  4%|▎         | 28/800 [00:05<03:12,  4.00it/s] 

Skipping run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4-cfr_10external_plus_linear-103 because of error None final checkpoint?
Skipping run feb4_3t_feb4_3t_2_base_dev1000_rho0_t3_tie_break-cfr_10external_plus_linear-103 because of error None final checkpoint?
Skipping run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4-cfr_10external_plus_linear-101 because of error None final checkpoint?
Skipping run feb4_3t_feb4_3t_2_base_dev1000_rho0_t3_tie_break-cfr_10external_plus_linear-102 because of error None final checkpoint?
Skipping run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4-cfr_10external_plus_linear-100 because of error None final checkpoint?
Skipping run feb4_3t_feb4_3t_2_base_dev1000_rho0_t3_tie_break-cfr_10external_plus_linear-104 because of error None final checkpoint?
Skipping run feb4_4t_feb4_4t_4_base_dev1000_rho1_t4-cfr_10external_plus_linear-102 because of error None final checkpoint?
Skipping run feb4_1t_feb4_1t_0_base_dev1000_rho0_t1-cfr_10external_plus_linear-104 because of error None fina

  5%|▍         | 39/800 [00:07<02:25,  5.23it/s]

KeyboardInterrupt



In [34]:
EquilibriumSolverRun.objects.filter(experiment__name='feb4_v1').count()

800

In [36]:
EquilibriumSolverRunCheckpoint.objects.filter(equilibrium_solver_run__experiment__name='feb4_v1').count()

400

In [40]:
run = EquilibriumSolverRun.objects.get(name="feb4_4t_feb4_4t_0_base_dev1000_rho1_t4_tie_break-cfr_10external_plus_linear-100")
run.equilibriumsolverruncheckpoint_set.count()

0

In [15]:
EquilibriumSolverRunCheckpoint.objects.last()

<EquilibriumSolverRunCheckpoint: feb4_2t_feb4_2t_1_base_dev1000_rho1_t2_tie_break-ppo_jun8_23ppo_76-103 (feb4_v1) Iteration 206848>

In [19]:
cp = EquilibriumSolverRunCheckpoint.objects.filter(equilibrium_solver_run__experiment__name="feb4_v1").first()

In [21]:
cp.get_modal_eval()

<Evaluation: Evaluation p0=modal+p1=modal for feb4_1t_feb4_1t_4_base_dev1000_rho0_t1_tie_break-ppo_jun8_23ppo_76-100 (feb4_v1) Iteration 190464>

In [24]:
cp_run = cp.equilibrium_solver_run

In [25]:
cp_run.equilibriumsolverruncheckpoint_set.all()

<QuerySet [<EquilibriumSolverRunCheckpoint: feb4_1t_feb4_1t_4_base_dev1000_rho0_t1_tie_break-ppo_jun8_23ppo_76-100 (feb4_v1) Iteration 190464>]>

In [32]:
list(Experiment.objects.get(name='feb4_v1').equilibriumsolverrun_set.values_list('name', flat=True))

['feb4_3t_feb4_3t_2_base_dev1000_rho0_t3_tie_break-cfr_10external_plus_linear-101',
 'feb4_1t_feb4_1t_0_base_dev1000_rho0_t1-cfr_10external_plus_linear-100',
 'feb4_4t_feb4_4t_4_base_dev1000_rho1_t4-cfr_10external_plus_linear-104',
 'feb4_1t_feb4_1t_0_base_dev1000_rho0_t1-cfr_10external_plus_linear-101',
 'feb4_1t_feb4_1t_1_base_dev1000_rho1_t1_tie_break-cfr_10external_plus_linear-104',
 'feb4_2t_feb4_2t_2_base_dev1000_rho1_t2-cfr_10external_plus_linear-100',
 'feb4_2t_feb4_2t_2_base_dev1000_rho1_t2-cfr_10external_plus_linear-101',
 'feb4_3t_feb4_3t_0_base_dev1000_rho1_t3-cfr_10external_plus_linear-100',
 'feb4_2t_feb4_2t_2_base_dev1000_rho1_t2-cfr_10external_plus_linear-102',
 'feb4_3t_feb4_3t_0_base_dev1000_rho1_t3-cfr_10external_plus_linear-101',
 'feb4_2t_feb4_2t_2_base_dev1000_rho1_t2-cfr_10external_plus_linear-103',
 'feb4_3t_feb4_3t_0_base_dev1000_rho1_t3-cfr_10external_plus_linear-102',
 'feb4_2t_feb4_2t_2_base_dev1000_rho1_t2-cfr_10external_plus_linear-104',
 'feb4_3t_feb4_3t_